# Part 1

# First we install beautifulsoup4

In [24]:
conda install -c anaconda beautifulsoup4

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


# Add python libraries

In [26]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Get table of postal codes from Wikipedia page

In [27]:
with open('List of postal codes of Canada.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')
table = soup.find('table')

# Transform the data into a pandas dataframe

In [28]:
table_rows = table.find_all('tr')

# get head of the table
table_head = table.find_all('th')
heads = [i.text for i in table_head]
heads[0] = 'PostalCode'
heads[-1] = heads[-1].strip()

# get content of the table
data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    if row:
        row[-1] = row[-1].strip()
        data.append(row)

#transform the table into a pandas dataframe
df = pd.DataFrame(data, columns = heads)
print(df.head())

  PostalCode           Borough     Neighbourhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront


# Delete the cells with a borough that is Not assigned

In [29]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace = True)
print(df.head())

  PostalCode           Borough     Neighbourhood
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
5        M5A  Downtown Toronto       Regent Park
6        M6A        North York  Lawrence Heights


# Merge rows having same postcode into one row with the neighborhoods separated with a comma

In [31]:
df1 = df.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ','.join(x))
df1.reset_index(level=['PostalCode','Borough'], inplace=True)
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


# Set borough with 'not assigned' in neighbourhood itself

In [32]:
df1['Neighbourhood'] = np.where(df1['Neighbourhood'] == 'Not assigned', df1['Borough'], df1['Neighbourhood'])
df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [33]:
df1.shape

(103, 3)

# Part 2

# Import csv file with geographical coordinates

In [34]:
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merge csv file with dataframe

In [36]:
#change df_geo columns name
df_geo.columns = ['PostalCode', 'Latitute','Longitude']

df_tor = pd.merge(df1, df_geo)
df_tor.head()

,PostalCode,Borough,Neighbourhood,Latitute,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
